In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# two intermediate layers network
class BasicNet(nn.Module):
    
    def __init__(self, num_features, num_classes=1):
        super().__init__()
        self.num_features = num_features
        # For your dataset this should be 1
        self.num_classes = num_classes
        self.n_layers = 4
        
        self.n_features = {1: num_features, 
                           2: int(num_features/2), 
                           3: int(num_features/4), 
                           4: num_classes}
        
        self.lin = {i : torch.nn.Linear(self.n_features[i], self.n_features[i+1]) 
                   for i in range(4)}

    def forward(self, x):
        for i in range(3):
            x = nn.relu(self.lin[i](x))
        # Don't use a ReLU at the end -- this need to be a number in [0,1]
        return nn.Sigmoid(self.lin[4](x))
            
    def get_embedding(self, x, k):
        assert k < 4
        for i in range(k):
            x = nn.relu(self.lin[i](x))
        return x

## Training the Basic Network

In [3]:
model = BasicNet(num_features, 1).to(device)
nbr_epochs = 10
lr = 0.0025
weight_decay = 0

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)

start_time = time.time()
for epoch in range(nbr_epochs):
    for (x, y) in pre_model.train_loader:
        (x, y) = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        ŷ = model(x) 

        loss = loss_fn(ŷ, y)
        loss.backward()
        optimizer.step()    
end_time = time.time()

NameError: name 'num_features' is not defined

In [5]:
class Cache:
    def __init__(self, n_layers):
        self.embeddings = {i: [] for i in range(n_layers)}
    @property
    def size(self):
        return len(self.embeddings)

    @property
    def get(self, i):
        assert i < len(self.embeddings)
        return self.embeddings[i]
    
    def store(self, x, i):
        self.embeddings[i].append(x)

### Store hidden representation in cache

In [7]:
cache = Cache(2) # we'll use only the hidden layers

for epoch in range(nbr_epochs):
    for (x, y) in pre_model.train_loader:
        model.step
        
        (x, y) = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        h2 = model.get_embedding(x, 2) 
        cache.store(h2, 2)
        
        h3 = model.get_embedding(x, 3) 
        cache.store(h3, 3)

NameError: name 'nbr_epochs' is not defined

In [9]:
class AutoEncoder(nn.Module):
    def __init__(self, indim, outdim):
        super(AutoEncoder, self).__init__()
        self.indim = indim
        self.outdim = outdim
        
        self.hidden_e = nn.Sequential(nn.Linear(indim, indim/2),
                                      nn.ReLU(),
                                      nn.Linear(int(indim/2), outdim))
        
        self.hidden_d = nn.Sequential(nn.Linear(outdim, int(indim/2)),
                                      nn.ReLU(),
                                      nn.Linear(int(indim/2), indim))
        
    def encode(self, x):
        return self.hidden_e(x)
    
    def decode(self, x):
        return self.hidden_d(x)
    
    def forward(self, x):
        return self.decode(self.encode(x))
    

## Train Autoencoder

In [ ]:
AE2 = AutoEncoder(model.n_features[2], 4) ## Not sure what's a good output dimensionality here (check based on your model)
AE3 = AutoEncoder(model.n_features[3], 4)

AE2_loss = nn.MSELoss()
AE2_opt = optim.Adam(AE2.parameters(), lr=0.001)

AE3_loss = nn.MSELoss()
AE3_opt = optim.Adam(AE3.parameters(), lr=0.001)

for e in range(n_epochs):
    for x in cache.embeddings[1]:
        #x_encoded = AE2.encode(x) 
        #x_decoded = AE2.encode(x_encoded)
        AE2_opt.zero_grad()
        x_decoded = AE2(x)
        l = AE2_loss(x_decoded, x)
        AE2_loss.backward()
        AE2_opt.step() 

    for x in cache.embeddings[2]:
        #x_encoded = AE3.encode(x) 
        #x_decoded = AE3.encode(x_encoded)
        AE3_opt.zero_grad()
        x_decoded = AE3(x)
        l = AE3_loss(x_decoded, x)
        AE3_loss.backward()
        AE3_opt.step() 